In [ ]:
import pandas as pd
import requests
from pathlib import Path
import numpy as np
import os
from bs4 import BeautifulSoup
import re

# Data Wrangling

1. Unify the file formats (i.e., `json` --> `csv`)
1. Merge metadata from multiple sources (i.e., NIH Common Fund repositories and Database Commons)
1. Add URLs of resources if missing (i.e., Journal homepages using `Sourceid`)
1. Add manually collected subpages (e.g., docs, search page, etc)
1. Add our own IDs for individual resources by updating (`global_data-portal_id_map.csv`)
<!-- 1. Add resource connection status -->

**To-Do**
- [ ] Merge NIH data portals

In [ ]:
EVALUATION_DATE_FOLDER = 'Nov-21-2023'

## Data Portals

In [ ]:
"""
Load the original data
"""
df = pd.read_json(os.path.join('../input', EVALUATION_DATE_FOLDER, 'database-commons.json'))

# df = df.head(10) # for testing purposes

In [ ]:
"""
We use underscore lowercase column names
"""
df.columns = (df.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower())

In [ ]:
"""
Drop columns that we don't need
"""
df.drop(columns=['biodb_ranks', 'rating_list'], inplace=True)

In [ ]:
"""
The id from the sources are consistently "source_id"
The values should be a string type, and it has the prefix that represents the source (e.g. dc_ for Database Commons)
"""
df.rename(columns={ "db_id": "source_id" }, inplace=True)
df.source_id = df.source_id.apply(lambda x: 'dc_' + str(x))

In [ ]:
"""
Some columns from data commons are in json format, we need to convert them to string
Example: [{ "id": 1, "name": "foo" }, { "id": 2, "name": "bar" }] --> 'foo, bar'
"""
json_column_names_and_keys = {
    'data_type_list': 'datatypeName', 
    'category_list': 'name',
    'keywords_list': 'name',
    'data_object_list': 'name',
    'organism_list': 'organismName',
    'theme_list': 'name'
}

for (column, key) in json_column_names_and_keys.items():
    df[column] = df[column].apply(lambda x: ', '.join([object[key] for object in x]))

In [ ]:
# This takes too much time. We will instead just run WAVE API requests for all URLs.
# # Check the connection status and put that as a `reachable` column

# # TODO: a faster way to do this?
# def check_connection_status(url):
#     print(url)
#     try:
#         status = requests.get(url)
#     except Exception:
#         return False
#     return status.status_code == 200
    
# df['reachable'] = df['url'].apply(lambda x: check_connection_status(x))

### Update the ID Mapping Table (`data-portal_id_map.csv`)

In [ ]:
"""
create or update a mapping table (i.e., `data-portal_id_map.csv`)
"""
# Load the data first
file = Path(os.path.join('../output', 'data-portal_id_map.csv'))
if file.exists():
    df_map = pd.read_csv(file)
else:
    df_map = pd.DataFrame(columns=['id', 'source_id', 'date_added'])

# Find rows that does not already exist in the table
df_temp = df_map.merge(df, how='outer', on='source_id', indicator=True)
df_new_resources = df_temp[df_temp['_merge'] == 'right_only'][['source_id']]

# Ensure to assign new `id`s, i.e., New `id` == max id + 1
max_id = df_map.id.max()
max_id = 0 if max_id is np.nan else max_id
new_id = max_id + 1

df_new_resources.insert(0, 'id', range(new_id, new_id + len(df_new_resources)))
df_new_resources['date_added'] = pd.to_datetime('today').strftime('%m-%d-%Y')

pd.concat([df_map, df_new_resources], axis=0).to_csv(file, index=False)

### Save the Metadata (`data-portal_metadata.csv`)

In [ ]:
"""
Add the `id` column to the original metadata
"""
df_map = pd.read_csv(file)
df_meta = df_map[['id', 'source_id']].merge(df, how='right', on='source_id')
df_meta.to_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'data-portal_metadata.csv'), index=False)
df_meta.head(1)

### Save Homepages (`data-portal_pages.csv`)

In [ ]:
"""
Create `data-portal_pages.csv` and add homepage urls.
"""
df_meta = pd.read_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'data-portal_metadata.csv'))

df_pages = df_meta[['id', 'url']].copy()

df_pages['page_type'] = 'home'
df_pages['page_id'] = df_pages['id']
df_pages['page_id'] = df_pages['page_id'].apply(lambda x: str(x) + '_' + 'home')

df_pages.to_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'data-portal_pages.csv'), index=False)

### Add Subpages

In [ ]:
"""
Add manually collected subpages to `data-portal_pages.csv` from `URL Collection for Subpages - Data Portals.csv`
"""
df_subpages = pd.read_csv(os.path.join('../input', EVALUATION_DATE_FOLDER, 'URL Collection for Subpages - Data Portals.csv'))

# add prefix to the source_id, following our naming convention
df_subpages.source_id = df_subpages.source_id.apply(lambda x: 'dc_' + str(x))

# Replace the descriptive page_type with the code
page_type_map = {
    'Home': 'home',
    'Search/Filter': 'search',
    'Search Result': 'search_result',
    'Data Entity (Detail Page)': 'data_entity',
    'Documentation or FAQ': 'documentation',
}
df_subpages.page_type = df_subpages.page_type.apply(lambda x: page_type_map[x])

# Append our `id`
df_map = pd.read_csv(os.path.join('../output', 'data-portal_id_map.csv'))
df_subpages = df_subpages.merge(df_map[['id', 'source_id']], how='left', on='source_id')

# Add `page_id` using both `id` and `page_type`
df_subpages['page_id'] = df_subpages['page_type'] + '_' + df_subpages['id'].astype(str)

# Load the existing pages
path_pages = os.path.join('../output', EVALUATION_DATE_FOLDER, 'data-portal_pages.csv')
df_pages = pd.read_csv(path_pages)

# Append the new pags
df_pages = df_pages.merge(df_subpages[['id', 'url', 'page_type', 'page_id']], how='outer', on=['page_id', 'page_type', 'id'])

# Now that we have two versions of URLs, we prefer to keep the manually collected ones (`url_y`) over the orignal ones (`url_x`) if exist
df_pages['url'] = df_pages.url_y.combine_first(df_pages.url_x)

# drop the temporary columns
df_pages.drop(columns=['url_x', 'url_y'], inplace=True)

# save the file
df_pages.to_csv(path_pages, index=False)

df_pages

## Journals

In [ ]:
"""
Load the original data
"""
df = pd.read_csv(os.path.join('../input', EVALUATION_DATE_FOLDER, 'scimagojr 2022.csv'), sep=';')

# df = df.head(5) # for testing purposes

In [ ]:
"""
Using `Sourceid` of SJR, get URLs of individual journal portals
TODO: Reuse the previously identified home pages
"""
def infer_homepage(Sourceid):
    info_url = f'https://www.scimagojr.com/journalsearch.php?q={Sourceid}&tip=sid&clean=0'
    html_text = requests.get(info_url).text
    soup = BeautifulSoup(html_text, 'html.parser')
    urls = soup.find_all('a', text=re.compile('Homepage'))
    if len(urls) > 0:
        return urls[0].get('href')
    else:
        print(f'No homepage found for {Sourceid}')
        return None

df['url'] = df['Sourceid'].apply(lambda x: infer_homepage(x))

In [ ]:
"""
We use underscore lowercase column names
"""
df.columns = (df.columns.str.replace('(?<=[a-z])(?=[A-Z])', '_', regex=True).str.lower())
df.columns = (df.columns.str.replace('.', '')) # remove dots
df.columns = (df.columns.str.replace('(', '')) # remove parentheses
df.columns = (df.columns.str.replace(')', ''))
df.columns = (df.columns.str.replace('/', 'per')) # replace slash with "per"
df.columns = (df.columns.str.replace(' ', '_')) # replace space with underscore

In [ ]:
"""
The id from the sources are consistently "source_id"
The values should be a string type, and it has the prefix that represents the source (e.g. dc_ for Database Commons)
"""
df.rename(columns={ "sourceid": "source_id" }, inplace=True)
df.source_id = df.source_id.apply(lambda x: 'sjr_' + str(x))

### Update the ID Mapping Table (`journal-portal_id_map.csv`)

In [ ]:
"""
create or update a mapping table (i.e., `data-portal_id_map.csv`)
"""
# Load the data first
file = Path(os.path.join('../output', 'journal-portal_id_map.csv'))
if file.exists():
    df_map = pd.read_csv(file)
else:
    df_map = pd.DataFrame(columns=['id', 'source_id', 'date_added'])

# Find rows that does not already exist in the table
df_temp = df_map.merge(df, how='outer', on='source_id', indicator=True)
df_new_resources = df_temp[df_temp['_merge'] == 'right_only'][['source_id']]

# Ensure to assign new `id`s, i.e., New `id` == max id + 1
max_id = df_map.id.max()
max_id = 0 if max_id is np.nan else max_id
new_id = max_id + 1

df_new_resources.insert(0, 'id', range(new_id, new_id + len(df_new_resources)))
df_new_resources['date_added'] = pd.to_datetime('today').strftime('%m-%d-%Y')

pd.concat([df_map, df_new_resources], axis=0).to_csv(file, index=False)

### Save the Metadata of Journal Portals (`journal-portal_metadata.csv`)

In [ ]:
"""
Add the `id` column to the original metadata
"""
df_map = pd.read_csv(file)
df_meta = df_map[['id', 'source_id']].merge(df, how='right', on='source_id')
df_meta.to_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'journal-portal_metadata.csv'), index=False)
df_meta.head(10)

### Save the Pages of Journal Portals (`journal-portal_pages.csv`)

In [ ]:
"""
Create `*_pages.csv` and add homepage urls.
"""
df_meta = pd.read_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'journal-portal_metadata.csv'))

df_pages = df_meta[['id', 'url']].copy()

df_pages['page_type'] = 'home'
df_pages['page_id'] = df_pages['id']
df_pages['page_id'] = df_pages['page_id'].apply(lambda x: str(x) + '_' + 'home')

df_pages.to_csv(os.path.join('../output', EVALUATION_DATE_FOLDER, 'journal-portal_pages.csv'), index=False)

### Add Subpages

In [ ]:
"""
Add manually collected subpages to `journal-portal_pages.csv` from `URL Collection for Subpages - Journals.csv`
"""
df_subpages = pd.read_csv(os.path.join('../input', EVALUATION_DATE_FOLDER, 'URL Collection for Subpages - Journals.csv'))

# add prefix to the source_id, following our naming convention
df_subpages.source_id = df_subpages.source_id.apply(lambda x: 'sjr_' + str(x))

# Replace the descriptive page_type with the code
page_type_map = {
    'Home': 'home',
    'Latest Open Access Research Article': 'research_article',
    'Latest None-research Article': 'none_research_article',
    'Article Search Result': 'article_search_result',
    'Latest Issue': 'latest_issue',
}
df_subpages.page_type = df_subpages.page_type.apply(lambda x: page_type_map[x])

# Append our `id`
df_map = pd.read_csv(os.path.join('../output', 'journal-portal_id_map.csv'))
df_subpages = df_subpages.merge(df_map[['id', 'source_id']], how='left', on='source_id')

# Add `page_id` using both `id` and `page_type`
df_subpages['page_id'] = df_subpages['page_type'] + '_' + df_subpages['id'].astype(str)

# Load the existing pages
path_pages = os.path.join('../output', EVALUATION_DATE_FOLDER, 'journal-portal_pages.csv')
df_pages = pd.read_csv(path_pages)

# Append the new pags
df_pages = df_pages.merge(df_subpages[['id', 'url', 'page_type', 'page_id']], how='outer', on=['page_id', 'page_type', 'id'])

# Now that we have two versions of URLs, we prefer to keep the manually collected ones (`url_y`) over the orignal ones (`url_x`) if exist
df_pages['url'] = df_pages.url_y.combine_first(df_pages.url_x)

# drop the temporary columns
df_pages.drop(columns=['url_x', 'url_y'], inplace=True)

# save the file
df_pages.to_csv(path_pages, index=False)

df_pages